In [56]:
!head -10 ml-1m/ratings.dat


1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


In [57]:
import pandas as pd
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter="::", engine="python", header=None)

In [58]:
ratings.columns = ["UserID::MovieID::Rating::Timestamp".split("::")]
#ratings = ratings.iloc[:1000, :]
ratings.head()


,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [59]:
ratings.isnull().sum()

UserID       0
MovieID      0
Rating       0
Timestamp    0
dtype: int64

In [60]:
import numpy as np
np.unique(ratings['Rating'])

array([1, 2, 3, 4, 5])

* Do we have any duplicate ratings, i.e., a (userid, movieid) combination that is repeated more than once?

In [61]:
user_movie_pairs = zip(ratings['UserID'].values, ratings['MovieID'].values)

In [62]:
print len(user_movie_pairs), len(set(user_movie_pairs))

1000209 1000209


In [63]:
print ratings['UserID'].unique()
set(ratings['UserID'].unique()) == set(range(1, 6041))
# User ids are range(1, 6041), i.e., are all values between 1 and 6040
# We can use a simple mapping: user_index=UserID-1
print "number of users: %s\n" % len(ratings['UserID'].unique())

[   1    2    3 ..., 6038 6039 6040]
number of users: 6040



In [64]:
!head ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


In [65]:
max(movies.MovieID)

3952

In [66]:
movies = pd.read_csv('ml-1m/movies.dat', delimiter='::', engine='python', header=None)
movies.columns = 'MovieID::Title::Genres'.split('::')
movies.head()

print "The following movie id's are missing from movie table"
print sorted(set(range(1, 3953)) - set(movies.MovieID))
print "\nnumber of unique movies: %s\n" %  len(set(movies.MovieID))

mi = ratings['MovieID'].unique()
mi.sort()
print sorted(set(movies.MovieID) - set(mi))
print len(mi)
# movie id have some missing values in addition to the missing values above
# , i.e., there are movies that are not rated by any user.

#movie_index_to_ID = dict(zip(range(movies.MovieID.unique().shape[0]), movies.MovieID))
movie_index_to_ID = dict(zip(range(len(mi)), mi))
#movie_ID_to_index = dict(zip(movies.MovieID, range(movies.MovieID.unique().shape[0])))
movie_ID_to_index = {k: v for v, k in movie_index_to_ID.iteritems()}


The following movie id's are missing from movie table
[91, 221, 323, 622, 646, 677, 686, 689, 740, 817, 883, 995, 1048, 1072, 1074, 1182, 1195, 1229, 1239, 1338, 1402, 1403, 1418, 1435, 1451, 1452, 1469, 1478, 1481, 1491, 1492, 1505, 1506, 1512, 1521, 1530, 1536, 1540, 1560, 1576, 1607, 1618, 1634, 1637, 1638, 1691, 1700, 1712, 1736, 1737, 1745, 1751, 1761, 1763, 1766, 1775, 1778, 1786, 1790, 1800, 1802, 1803, 1808, 1813, 1818, 1823, 1828, 1838, 3815]

number of unique movies: 3883

[51, 109, 115, 143, 284, 285, 395, 399, 400, 403, 604, 620, 625, 629, 636, 654, 675, 676, 683, 693, 699, 713, 721, 723, 727, 738, 739, 752, 768, 770, 772, 773, 777, 794, 795, 797, 812, 816, 819, 822, 825, 845, 855, 856, 857, 871, 873, 890, 894, 979, 983, 1001, 1045, 1052, 1065, 1075, 1106, 1108, 1109, 1110, 1122, 1137, 1140, 1141, 1143, 1146, 1155, 1156, 1157, 1158, 1159, 1166, 1308, 1309, 1314, 1318, 1319, 1368, 1400, 1424, 1443, 1448, 1462, 1467, 1524, 1557, 1559, 1568, 1577, 1578, 1628, 1697, 1698, 1705,

In [67]:
len(movies.MovieID) == len(movies.MovieID.unique())

True

In [68]:
from scipy.sparse import csr_matrix
row = ratings['MovieID'].values
col = ratings['UserID'].values
data = ratings['Rating'].values
ratings_matrix = csr_matrix((data, (row, col)))



In [69]:
ratings_matrix.shape

(3953, 6041)

In [70]:
def trace_dot(X, Y):
    """Trace of np.dot(X, Y.T)."""
    return np.dot(X.ravel(), Y.ravel())
norm_X = np.dot(X.data, X.data)
norm_WH = trace_dot(np.dot(np.dot(W.T, W), H), H)
cross_prod = trace_dot((X * H.T), W)
error = np.sqrt(norm_X + norm_WH - 2. * cross_prod)

NameError: name 'X' is not defined

In [ ]:
error/X.shape[0]/X.shape[1]

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("WARN")


In [71]:
from pyspark.sql.functions import rand

def kfold_cv_generator(dataset, n_folds=3, seed = None):
    if not seed:
        seed = 0
    h = 1.0 / n_folds
    randCol = "cv_rand"
    df = dataset.select("*", rand(seed).alias(randCol)).cache()
    for i in range(n_folds):
        validateLB = i * h
        validateUB = (i + 1) * h
        condition = (df[randCol] >= validateLB) & (df[randCol] < validateUB)
        validation = df.filter(condition)
        train = df.filter(~condition)
        yield train.drop(randCol), validation.drop(randCol)

In [72]:
#ratings_sp = spark.createDataFrame(ratings)

In [73]:
ratings.columns

Index([u'UserID', u'MovieID', u'Rating', u'Timestamp'], dtype='object')

In [74]:
ratings_sp.columns

['UserID', 'MovieID', 'Rating', 'Timestamp']

In [75]:
import os
#TODO: Should use broadcast for movie_ID_to_index?
def parse_data(line):
    fields = line.split("::")
    user_id = int(fields[0])
    movie_id = int(fields[1])
    rating = float(fields[2])
    ts = fields[3]
    return movie_ID_to_index_bc.value[movie_id], user_id - 1, rating, ts
    
movie_ID_to_index_bc = sc.broadcast(movie_ID_to_index)
u_data_file = os.path.join("ml-1m", "ratings.dat")
ratings_rdd = sc.textFile(u_data_file).map(parse_data)#.cache().filter(lambda x: x is not None)

In [76]:
x = ratings_rdd.collect()
pd.DataFrame(x).describe()

,0,1,2
count,1.000209e+06,1.000209e+06,1.000209e+06
mean,1.730481e+03,3.023512e+03,3.581564e+00
std,1.017026e+03,1.728413e+03,1.117102e+00
min,0.000000e+00,0.000000e+00,1.000000e+00
25%,9.650000e+02,1.505000e+03,3.000000e+00
50%,1.657000e+03,3.069000e+03,4.000000e+00
75%,2.565000e+03,4.475000e+03,4.000000e+00
max,3.705000e+03,6.039000e+03,5.000000e+00


In [ ]:
number of unique movies: 3883
number of users: 6040

In [77]:
ratings_columns = "UserID::MovieID::Rating::Timestamp".split("::")
ratings_sp = spark.createDataFrame(ratings_rdd, schema=ratings_columns)

In [ ]:
import numpy as np
import os
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

#u_data_file = os.path.join("ml-1m", "ratings.dat")
#ratings = sc.textFile(u_data_file).map(parse_data).cache()#.filter(lambda x: x is not None)

#ratings_train = ratings.filter(lambda x: x[3] < 8).map(lambda x: x[:-1])
#ratings_test = ratings.filter(lambda x: x[3] >= 8).map(lambda x: x[:-1])

#print("number of train points: %d" % ratings_train.count())
#print("number of test points: %d" % ratings_test.count())

n_folds = 10
MSE = np.zeros(n_folds)
for i, (ratings_train, ratings_test) in enumerate(kfold_cv_generator(ratings_sp, n_folds)):
    
    print ratings_train.count()
    # TODO: Remove timestamp column from the begining
    model = ALS.train(ratings_train.select(['UserID', 'MovieID', 'Rating']), rank = 10, iterations = 10)

    test_data = ratings_test.rdd.map(lambda r: (r[0], r[1]))
    predictions = model.predictAll(test_data).map(lambda r: ((r[0], r[1]), r[2]))
    p = predictions.collect()
    ratesAndPreds = ratings_test.rdd.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
    MSE[i] = (ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean())**.5

    print("Mean Squared Error = " + str(MSE))


In [78]:
model = ALS.train(ratings_sp.select(['UserID', 'MovieID', 'Rating']), rank = 10, iterations = 10)

In [79]:
pf = model.productFeatures().collect()

In [80]:
uf = model.userFeatures().collect()

In [ ]:
from matplotlib import pyplot as plt

In [93]:
movie_index_list = [i[1] for i in model.recommendProducts(0, 10)]
[movie_index for movie_index_to_ID[movie_index]

TypeError: unhashable type: 'list'

In [ ]:
def coordinateMatrixMultiply(leftMatrix, rightMatrix)
  M_ = leftMatrix.map(lambda r: (r[0], (r[1], r[2])))
  N_ = rightMatrix.map(lambda r: (r[0], (r[1], r[2])))

  M_.join(N_)
    .map({ case (_, ((i, v), (k, w))) => ((i, k), (v * w)) })
    .reduceByKey(_ + _)
    .map({ case ((i, k), sum) => MatrixEntry(i, k, sum) })

  new CoordinateMatrix(productEntries)
}

In [ ]:
number of unique movies: 3883
number of users: 6040

In [ ]:
set(x) == set(range(6040))

In [ ]:
set(y) == set(range(3706))

In [ ]:
x = [i[0] for i in pf]

In [ ]:
y = [i[0] for i in uf]

In [ ]:
pd.DataFrame(x).describe()

In [ ]:
max(sorted(y))

In [ ]:
pd.DataFrame(y).describe()

In [ ]:
plt.plot(sorted(x), '.')
plt.show()

In [ ]:
dir(model)

In [ ]:
np.mean([i[1] for i in p])

In [ ]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder



lr = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, np.logspace(-2, 2, 5)) \
    .build()

cnt = 0

class Evaluator:
    def evaluate(self, dataset):
        global cnt
        cnt += 1
        print dataset.count()
        print cnt
        #print dataset['features'].collect()
        #print dataset['label']
        print
        return 1
    
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          #evaluator=BinaryClassificationEvaluator(),
                          evaluator=Evaluator(),
                          numFolds=3)  # use 3+ folds in practice

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(training)



In [ ]:
import os
spark_home = os.environ['SPARK_HOME']
training = spark.read.format("libsvm").load(os.path.join(spark_home, "data/mllib/sample_libsvm_data.txt"))


In [ ]:
training